In [22]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import torch.nn.functional as F
from tqdm import tqdm
import copy
import collections

In [15]:
def gen_data(n, d, p, w, gamma=1):
    # sample labels from torch where label 1 has probability p
    ys = torch.distributions.Bernoulli(torch.tensor(p)).sample((n, 1))
    # sample points from a d dimensional gaussian
    zs = torch.randn(n, d)
    xs = zs + gamma * ys * w
    return xs.float(), ys

In [16]:
def bayes_accuracy(xTrain, yTrain, xTest, yTest):
    model = nn.Sequential(nn.Linear(xTrain.shape[1], 128), nn.ReLU(), nn.Linear(128, 128), nn.ReLU(), nn.Linear(128, 1), nn.Sigmoid())
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    for i in tqdm(range(10000)):
        optimizer.zero_grad()
        loss = F.binary_cross_entropy(model(xTrain), yTrain)
        loss.backward()
        optimizer.step()
    return ((model(xTest) > 0.5) == yTest).float().mean().item()


In [17]:
def train_on_source(xTrain, yTrain, xTest, yTest):
    model = nn.Sequential(nn.Linear(xTrain.shape[1], 1), nn.Sigmoid())
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    for i in tqdm(range(10000)):
        optimizer.zero_grad()
        loss = F.binary_cross_entropy(model(xTrain), yTrain)
        loss.backward()
        optimizer.step()
    print("\n Accuracy is:", ((model(xTest) > 0.5) == yTest).float().mean().item())
    return model

In [ ]:
def adapt(model, xTarget):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for i in tqdm(range(1000)):
        optimizer.zero_grad()
        probPreds = model(xTarget)
        labelPreds = (probPreds > 0.5).float().detach()
        loss = F.binary_cross_entropy(probPreds, labelPreds)
        loss.backward()
        optimizer.step()
    return model

In [19]:
def evaluate(model, x, y):
    return((model(x) > 0.5) == y).float().mean().item()

In [20]:
xSourceTrain, ySourceTrain = gen_data(1000, 10, 0.5, np.ones(10)/np.sqrt(10))
xSourceTest, ySourceTest = gen_data(1000, 10, 0.5, np.ones(10)/np.sqrt(10))

p = 0.75
xTargetTrain, yTargetTrain = gen_data(1000, 10, p, np.ones(10)/np.sqrt(10))
xTargetTest, yTargetTest = gen_data(1000, 10, p, np.ones(10)/np.sqrt(10))

In [ ]:
bAccuSource = bayes_accuracy(xSourceTrain, ySourceTrain, xSourceTest, ySourceTest)
bAccuTarget = bayes_accuracy(xTargetTrain, yTargetTrain, xTargetTest, yTargetTest)
print("Bayes accuracy on source is:", bAccuSource)
print("Bayes accuracy on target is:", bAccuTarget)

In [ ]:
model = train_on_source(xSourceTrain, ySourceTrain, xSourceTest, ySourceTest)
print("Accuracy on source is:", evaluate(model, xSourceTest, ySourceTest))
print("Accuracy on target is:", evaluate(model, xTargetTest, yTargetTest))
model = adapt(model, xTargetTrain)
print("Accuracy on source is:", evaluate(model, xSourceTest, ySourceTest))
print("Accuracy on target is:", evaluate(model, xTargetTest, yTargetTest))

In [ ]:
bayesAccus = collections.defaultdict(int)
for _ in range(10):
    for i in range(10):
        p = 0.5 + i/20
        xTargetTrain, yTargetTrain = gen_data(1000, 10, p, np.ones(10)/np.sqrt(10))
        xTargetTest, yTargetTest = gen_data(1000, 10, p, np.ones(10)/np.sqrt(10))
        bAccuTarget = bayes_accuracy(xTargetTrain, yTargetTrain, xTargetTest, yTargetTest)
        bayesAccus[p] += bAccuTarget
for p in bayesAccus:
    bayesAccus[p] /= 10

In [ ]:
plt.plot(list(bayesAccus.keys()), list(bayesAccus.values()), label="Bayes")

In [ ]:
accus = collections.defaultdict(int)
for _ in range(10):
    xSourceTrain, ySourceTrain = gen_data(1000, 10, 0.5, np.ones(10)/np.sqrt(10))
    xSourceTest, ySourceTest = gen_data(1000, 10, 0.5, np.ones(10)/np.sqrt(10))
    model = train_on_source(xSourceTrain, ySourceTrain, xSourceTest, ySourceTest)
    for i in range(10):
        temp_model = copy.deepcopy(model)
        p = 0.5 + i/20
        xTargetTrain, yTargetTrain = gen_data(1000, 10, p, np.ones(10)/np.sqrt(10))
        xTargetTest, yTargetTest = gen_data(1000, 10, p, np.ones(10)/np.sqrt(10))
        temp_model = adapt(temp_model, xTargetTrain)
        accus[p] += evaluate(temp_model, xTargetTest, yTargetTest)
for p in accus:
    accus[p] /= 10

In [ ]:
plt.plot(list(bayesAccus.keys()), list(bayesAccus.values()), label="Bayes")
plt.plot(list(accus.keys()), list(accus.values()), label="NN")
plt.xlabel("p")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
plt.plot(np.arange(0.5, 1, 0.05), accus, label="NN")
plt.plot(np.arange(0.5, 1, 0.05), bayesAccus, label="Bayes")
    plt.xlabel("p")
    plt.ylabel("Accuracy")
    plt.legend()
